In [2]:
import pandas as pd

In [3]:
#This is the 2 million dataset
data = pd.read_csv('user_locations.csv')
#ASSUMPTION: I'm not sure whether this dataset also contains the geotagged tweets. Specific geolocation is better, but user_location can serve as a heuristic
#However, I'm neglecting the coordinates that exist because just looking at user_location worked better for Tableau purposes
data=data.drop(['coordinates','place'], axis=1)

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#Conventional format for the user_location is City, STATE. This split city and state (for most entries)
location=data["user_location"].str.split(",", n = 1, expand = True) 
place=location[0]
state=location[1]
data["place"]=place
data['state']=state
#convert all states to upper case, such that states will match later on
data['state']=data['state'].str.upper() 


In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,created_at,id,text,lang,user_location,place,state
0,0,0,Thu May 07 11:56:39 +0000 2020,1258365124517203969,This is a sad reality. AmeriKKKa must realize ...,en,"Detroit, MI",Detroit,MI
1,1,1,Thu May 07 11:56:42 +0000 2020,1258365136999518208,RT @JhadiyahG: Please Share Ahmaud Arbery was ...,en,"Philadelphia, PA",Philadelphia,PA
2,2,2,Thu May 07 11:57:00 +0000 2020,1258365210827644930,RT @SharLovegrove: DEVASTATED by the @guardian...,en,"Crazy Horse, ND",Crazy Horse,ND
3,3,3,Thu May 07 11:57:05 +0000 2020,1258365231497203714,RT @ZayyWhosane: This shit so frustrating #Bla...,en,"New Orleans, LA",New Orleans,LA
4,4,4,Thu May 07 11:57:23 +0000 2020,1258365308751970304,We can't break down on a public road with car ...,en,"Mequon, WI",Mequon,WI


In [8]:
#This shows that there are still many with another comma and the (likely) true state behind it
data['state'].value_counts()

 CA                     381101
 TX                     230524
 NY                     167879
 FL                     117951
 IL                      87870
                         ...  
 REFERENDUMSHIRE, UK         1
 RI•ENFIELD, CT              1
 CA ✈️ PLANO, TX             1
 DC / AURORA, NY             1
 LA // EULESS, TX            1
Name: state, Length: 7138, dtype: int64

In [ ]:
nostates=data['state'].value_counts()
nostates.tail(50) #This shows the weird states the previous part tries to get out

In [23]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"] #DC is the 51st state

data['state2']=0

n=-1
for i in data['state']:
    n=n+1
    for j in states:
        if str(i).endswith(j): # ending with the correct state||.str excepts the AttributeError
            data['state2'][n]=j
        else:
            continue
#ASSUMPTION: I'm assuming that the ending state mentioned is the true state a person is in. Of course this does not have to be 
#the case. However, we are looking at city + state Tableau will not be able to locate all the cities which are not in the 
#state given. Therefore Tableau in a way checks of city + state do exist.
#ASSUMPTION: Likewise I'm assuming that the city is mentioned first, this might neclect a more specific street adress given.

<ipython-input-23-3b0b31a4db93>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['state2'][n]=j
C:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [24]:
data

,Unnamed: 0,Unnamed: 0.1,created_at,id,text,lang,user_location,place,state,state2
0,0,0,Thu May 07 11:56:39 +0000 2020,1258365124517203969,This is a sad reality. AmeriKKKa must realize ...,en,"Detroit, MI",Detroit,MI,MI
1,1,1,Thu May 07 11:56:42 +0000 2020,1258365136999518208,RT @JhadiyahG: Please Share Ahmaud Arbery was ...,en,"Philadelphia, PA",Philadelphia,PA,PA
2,2,2,Thu May 07 11:57:00 +0000 2020,1258365210827644930,RT @SharLovegrove: DEVASTATED by the @guardian...,en,"Crazy Horse, ND",Crazy Horse,ND,ND
3,3,3,Thu May 07 11:57:05 +0000 2020,1258365231497203714,RT @ZayyWhosane: This shit so frustrating #Bla...,en,"New Orleans, LA",New Orleans,LA,LA
4,4,4,Thu May 07 11:57:23 +0000 2020,1258365308751970304,We can't break down on a public road with car ...,en,"Mequon, WI",Mequon,WI,WI
...,...,...,...,...,...,...,...,...,...,...
2088217,914745,914744,Sun Jun 14 00:18:06 +0000 2020,1271960067785371650,RT @JoshuaPotash: The Paris #BlackLivesMatter ...,en,"Los Angeles, CA",Los Angeles,CA,CA
2088218,914746,914745,Sun Jun 14 00:18:07 +0000 2020,1271960069387825152,RT @JoshuaPotash: It seems like we're seeing l...,en,"Joplin, MO",Joplin,MO,MO
2088219,914747,914746,Sun Jun 14 00:18:07 +0000 2020,1271960070641704960,RT @JoshuaPotash: The Paris #BlackLivesMatter ...,en,"Costa Mesa, CA",Costa Mesa,CA,CA
2088220,914748,914747,Sun Jun 14 00:18:07 +0000 2020,1271960071153414144,RT @JoshuaPotash: The Paris #BlackLivesMatter ...,en,"Long Beach, CA",Long Beach,CA,CA


In [25]:
nostates=data['state2'].value_counts() #Also shows 0, which consists of UK and all the wrong values

CA    383149
TX    232644
NY    169450
FL    118819
0      88874
IL     88405
GA     86577
WA     74515
PA     62520
NC     51130
OH     47081
MA     45580
DC     44966
AZ     41464
MI     41343
VA     36578
OR     36458
MD     36118
TN     34955
NV     31433
CO     29886
LA     29749
NJ     26285
MN     25136
MO     23864
IN     22017
WI     15168
KY     15110
AL     14939
SC     14786
OK     13841
CT     10835
HI      9454
UT      9419
IA      8580
KS      7976
NM      7655
MS      6681
AR      6305
NE      6178
RI      5034
ID      4232
AK      3680
WV      3547
DE      2864
NH      2733
ME      2665
MT      2439
VT      1575
SD      1371
ND      1241
WY       918
Name: state2, dtype: int64

In [26]:

print("length before:", len(data))

data2= data[data['state2'] != 0] #Drop the "non-United-States", 88874 entries
print("length after:", len(data2))

data2=data2.drop(['state'], axis=1) #Duplicate
data2.to_csv('userloctationCOMPLETE.csv') #https://amsuni-my.sharepoint.com/:f:/g/personal/elise_mertens_student_uva_nl/EiDO45mW2X5Jnjgl2SObG7YBGID5hBuKEDxo2AV4Zia6HA?e=eLPOSz

length before: 2088222
length after: 1999348


In [38]:
#Playing around
datajun5=data2[data2['created_at'].str.contains('Jun 05')]
datajun6=data2[data2['created_at'].str.contains('Jun 06')]
datajun7=data2[data2['created_at'].str.contains('Jun 07')]
print(datajun5['place'].value_counts().head(20),"\n",datajun6['place'].value_counts().head(20),"\n",datajun7['place'].value_counts().head(20))

Los Angeles      8784
Chicago          5143
Washington       5089
New York         4801
Houston          4603
Atlanta          4340
Dallas           2871
Brooklyn         2639
San Diego        2551
Seattle          2536
Philadelphia     2339
Boston           2157
Austin           2156
Miami            1994
Portland         1963
San Francisco    1885
Las Vegas        1727
San Antonio      1404
Orlando          1344
Baltimore        1322
Name: place, dtype: int64 
 Los Angeles      9324
Chicago          5565
Houston          5150
Atlanta          4849
New York         4775
Washington       4419
Dallas           3301
Seattle          2946
Brooklyn         2806
Philadelphia     2498
San Diego        2496
Miami            2286
Portland         2126
San Francisco    2102
Austin           2087
Boston           2073
Las Vegas        1889
San Antonio      1567
Phoenix          1525
Orlando          1521
Name: place, dtype: int64 
 Los Angeles      7551
Chicago          4102
New York         390